In [1]:
import sys

sys.path.append("/Users/kanishkjain/opt/anaconda3/envs/gym/lib/python3.9/site-packages")

import random
import collections
from pprint import pprint

import gym
import gym_toytext
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# class Agent:
#     def __init__(
#         self, environment="Roulette-v0", gamma=0.1, theta=1e-6, epsilon=1.0, alpha=0.1
#     ) -> None:

#         self.env = gym.make(environment)
#         self.env.reset()

#         self.gamma = gamma
#         self.theta = theta
#         self.epsilon = epsilon

#         self.alpha = alpha

#         self.A_space = self.env.action_space
#         self.S_space = self.env.observation_space
#         self.R_range = self.env.reward_range

#         self.Num_A = self.A_space.n
#         self.Num_S = self.S_space.n
        
#         if environment == 'CliffWalking-v0':
#             for s in range(self.Num_S):
#                 for a in range(self.Num_A):
#                     P, S_, R_, T = self.env.P[s][a][0]
#                     if T:
#                         self.env.P[s][a] = [(P, S_, 0, T)]

#     def soft_policy(self):

#         Pi = np.ones((self.Num_S, self.Num_A)) / self.Num_A
#         return Pi

#     def greedy_policy(self, Q):
#         Pi = np.zeros((self.Num_S, self.Num_A))
#         for s in range(self.Num_S):
#             a_star = np.argmax([Q[(s, a)] for a in range(self.Num_A)])
#             Pi[s, a_star] = 1.0
#         return Pi

#     def epsilon_greedy_policy(self, Q, s):

#         p = random.random()
#         if p < self.epsilon:
#             return np.random.choice(self.Num_A)
#         else:
#             # A = np.argmax([Q[(s, a)] for a in range(self.Num_A)])
#             # prob = np.array([np.exp(x) for x in Q[s]])
#             # prob = prob/(sum(prob))
#             # A = np.random.choice(self.Num_A, p=prob)
#             A = np.argmax(Q[s])
#             return A

#     def on_policy_monte_carlo(self, num_iter=10):

#         self.epsilon = 0.9

#         NUM_ITER = num_iter

#         Q = collections.defaultdict(lambda: np.ones(self.Num_A)/self.Num_A)
#         returns = collections.defaultdict(float)

#         Pi = self.soft_policy()
#         print("Starting Policy:, ", Pi)

#         S_A_count = collections.defaultdict(int)

#         rewards_per_episode = []
#         unique_states = []

#         for it in range(NUM_ITER):
#             episode = self.generate_episode(Pi)
#             if it % 50 == 0:
#                 print(f"Generating Episode Number: {it}")

#             # self.epsilon = max(self.epsilon * 0.99, 0.01)

#             S_A = set([(S, A) for (S, A, _) in episode])

#             for S, A in S_A:
#                 first_idx = [
#                     i for i, (s, a, _) in enumerate(episode) if (s == S and a == A)
#                 ][0]
#                 G = sum(
#                     [
#                         r * (self.gamma ** i)
#                         for i, (s, a, r) in enumerate(episode[first_idx:])
#                     ]
#                 )

#                 returns[(S, A)] += G
#                 S_A_count[(S, A)] += 1
#                 Q[S][A] = returns[(S, A)] / S_A_count[(S, A)]

#             distinct_states = set([s for s, a in S_A])

#             for s in distinct_states:
#                 a_star = np.argmax(Q[s])
#                 for a in range(self.Num_A):
#                     if a == a_star:
#                         Pi[s][a] = 1 - self.epsilon + self.epsilon / self.Num_A
#                     else:
#                         Pi[s][a] = self.epsilon / self.Num_A

#         return Pi

#     def off_policy_monte_carlo(self, num_iter=10):

#         self.epsilon = 0.3

#         NUM_ITER = num_iter

#         Q = collections.defaultdict(lambda: np.ones(self.Num_A)/self.Num_A)
#         C = collections.defaultdict(lambda: np.zeros(self.Num_A))
        
#         Mu = self.soft_policy()

#         for it in range(NUM_ITER):
#             episode = self.generate_episode(Mu)
#             if it % 50 == 0:
#                 print(f"Generating Episode Number: {it}")

#             G = 0
#             W = 1
            
#             state = set()
#             for it_, info in enumerate(episode[::-1]):
#                 S, A, R = info
                
#                 state.add(S)
                
#                 G = self.gamma * G + R
                
#                 C[S][A] += W
#                 Q[S][A] = Q[S][A] + (W/C[S][A]) * (G - Q[S][A])

#                 Pi = self.epsilon_greedy_policy(Q, S)
                
#                 W = W * Pi / Mu[S, A]
                
#                 if W == 0:
#                     print(S, A, R, it_)
#                     break

#             print(state)
#         pprint(Q)
#         return Q

#     def q_learning(self, num_iter=301):

#         self.epsilon = .9

#         NUM_ITER = num_iter

#         Q = collections.defaultdict(lambda: np.zeros(self.Num_A))

#         for it in range(NUM_ITER):

#             if it % 50 == 0:
#                 print(f"Generating Episode Number: {it}")

#             S = self.env.reset()

#             # self.epsilon = max(self.epsilon * 0.999, 0.1)

#             while True:
#                 A = self.epsilon_greedy_policy(Q, S)
#                 S_, R, terminal, _ = self.env.step(A)

#                 Q[S][A] += self.alpha * (R + self.gamma * max(Q[S_]) - Q[S][A])

#                 S = S_

#                 if terminal:
#                     break

#         return Q

#     def sarsa(self, num_iter=301):

#         self.epsilon = .9

#         NUM_ITER = num_iter

#         Q = collections.defaultdict(lambda: np.zeros(self.Num_A))

#         for it in range(NUM_ITER):
#             if it % 50 == 0:
#                 print(f"Generating Episode Number: {it}")

#             S = self.env.reset()
#             A = self.epsilon_greedy_policy(Q, S)

#             #self.epsilon = max(self.epsilon * 0.999, 0.1)

#             while True:
#                 S_, R, terminal, _ = self.env.step(A)
#                 A_ = self.epsilon_greedy_policy(Q, S_)
                
#                 Q[S][A] += self.alpha*(R + (self.gamma * Q[S_][A_]) - Q[S][A])

#                 S = S_
#                 A = A_

#                 if terminal:
#                     break
#         return Q

#     def generate_episode(self, Pi):

#         episode = []

#         S = self.env.reset()
#         while True:
#             A = np.random.choice(np.arange(self.Num_A), p=Pi[S])
#             S_, R, terminal, _ = self.env.step(A)
#             episode.append((S, A, R))
#             S = S_
#             if terminal:
#                 break
#         return episode

#     def show_policy(self, Q):

#         MAX_STEPS = 500

#         S = self.env.reset()
#         print(f"Starting state: {S}")
#         # self.env.render()

#         step = 0
#         while step < MAX_STEPS:
#             # A = self.epsilon_greedy_policy(Q, S)
#             A = np.argmax(Q[S])
#             S_, R, done, _ = self.env.step(A)
#             # self.env.render()
#             if done:
#                 break
#             print(
#                 f"Current State: {S}, action: {A}, reward: {R}, done: {done}, step: {step}"
#             )
#             S = S_
#             step += 1
#         print(
#             f"Current State: {S}, action: {A}, reward: {R}, done: {done}, step: {step}"
#         )
#         # self.env.render()
#         self.env.close()
#         print("Finished", done)

In [12]:
class Agent:
    def __init__(
        self, environment="Taxi-v3"
    ) -> None:

        self.environment = environment

    def epsilon_greedy_policy(self, Pi, epsilon):
        
        Pi = np.around(Pi, decimals=1)

        p = np.random.rand()
        Num_A = len(Pi)
        if p < epsilon:
            return np.random.choice(Num_A)
        else:
            a_star = np.max(Pi)
            max_indices = [i for i in range(Num_A) if Pi[i]==a_star]
            A = np.random.choice(max_indices)
            return A
    
    def soft_policy(self, Num_A):
        A = np.random.choice(Num_A)
        return A
    
    def on_policy_monte_carlo(self, num_iter=2000, alpha=0.5,  gamma=0.6, epsilon = 0.7):

        env = gym.make(self.environment)
        Num_A = env.action_space.n
        Num_S = env.observation_space.n
        env.reset()

        NUM_ITER = num_iter

        Q = np.zeros((Num_S, Num_A))
        C = np.zeros((Num_S, Num_A))
        rewards = np.zeros(NUM_ITER)

        Pi = np.ones((Num_S, Num_A))/Num_A
        
        returns = collections.defaultdict(list)

        for it in range(1, NUM_ITER + 1):
            if it % 100 == 0:
                print(f"Generating Episode Number: {it}")
                
            S = env.reset()
            
            G = 0
            step = 0
            episode = []
            while True:
                if np.random.rand() < epsilon:
                    A = np.random.choice(Num_A)
                else:
                    a_star = np.max(Pi[S])
                    max_indices = [i for i in range(Num_A) if Pi[S, i]==a_star]
                    A = np.random.choice(max_indices)
            
                S_, R, terminal, _ = env.step(A)
                
                if terminal and self.environment == 'CliffWalking-v0':
                    R = 0
                
                episode.append((S, A, R))
                
                G = G + (gamma**step)*R

                S = S_
                
                step+=1

                if terminal:
                    break
                    
            rewards[it - 1] = G

            S_A = [(S, A) for (S, A, _) in episode]
            
            G = 0
            t = len(episode) - 1
            while t >= 0:
                S, A, R = episode[t]
                G = G + R
                if not (S, A) in S_A[:t]:
                    returns[(S, A)].append(G)
                    Q[S, A] = sum(returns[(S, A)])/len(returns[(S, A)])
                    
                    q_max = np.max(Q[S])
                    max_indices = [i for i in range(Num_A) if Q[S, i]==q_max]
                    A_star = np.random.choice(max_indices)
                    
                    for a in range(Num_A):
                        if a == A_star:
                            Pi[S, a] = 1 - epsilon + (epsilon/Num_A)
                        else:
                            Pi[S, a] = (epsilon/Num_A)
                t-=1
                
        return Q, rewards, Pi
    
    def off_policy_monte_carlo(self, num_iter=500, alpha=0.1, gamma=0.9, epsilon = 0.2):

        env = gym.make(self.environment)
        Num_A = env.action_space.n
        Num_S = env.observation_space.n
        env.reset()

        NUM_ITER = num_iter

        # Q = np.random.rand(Num_S, Num_A)
        Q = np.zeros((Num_S, Num_A))
        C = np.zeros((Num_S, Num_A))
        rewards = np.zeros(NUM_ITER)
        
        Pi = np.ones((Num_S, Num_A))/Num_A

        for it in range(1, NUM_ITER + 1):
            if it % 100 == 0:
                print(f"Generating Episode Number: {it}")

            S = env.reset()
            
            G = 0
            step = 0
            episode = []
            while True:    
                A = np.random.choice(Num_A)
                S_, R, terminal, _ = env.step(A)
                
                if terminal and self.environment == 'CliffWalking-v0':
                    R = 0
                
                episode.append((S, A, R))
                
                G = G + (gamma**step)*R

                S = S_
                
                step+=1

                if terminal:
                    break
                    
            rewards[it - 1] = G
            
            G = 0
            W = 1
            t = len(episode) - 1
            while t >= 0:
                S, A, R = episode[t]
                
                G = gamma * G + R
                
                C[S, A] += W
                Q[S, A] += (W/C[S, A]) * (G - Q[S, A])
                
                q_max = np.max(Q[S])
                max_indices = [i for i in range(Num_A) if Q[S, i]==q_max]
                A_star = np.random.choice(max_indices)
                
                for a in range(Num_A):
                    if a == A_star:
                        Pi[S, a] = 1 - epsilon + (epsilon/Num_A)
                    else:
                        Pi[S, a] = (epsilon/Num_A)
                
                W = W*(Pi[S, A]*Num_A)
                
                if W == 0:
                    break
                
                t -= 1

        return Q, rewards
    
    def q_learning(self, num_iter=10000, alpha=0.1, gamma=0.6, epsilon = 0.3):

        env = gym.make(self.environment)
        Num_A = env.action_space.n
        Num_S = env.observation_space.n
        env.reset()

        NUM_ITER = num_iter

        Q = np.zeros((Num_S, Num_A))
        rewards = np.zeros(NUM_ITER)

        for it in range(1, NUM_ITER + 1):

            if it % 5000 == 0:
                print(f"Generating Episode Number: {it}")

            S = env.reset()
            
            G = 0
            step = 0
            while True:
                A = self.epsilon_greedy_policy(Q[S], epsilon)
                S_, R, terminal, _ = env.step(A)
                
                G = G + (gamma**step)*R

                Q[S][A] += alpha * (R + gamma * np.max(Q[S_]) - Q[S][A])

                S = S_
                
                step+=1

                if terminal:
                    break
                    
            rewards[it -1] = G

        return Q, rewards

    def sarsa(self, num_iter=10000, alpha=0.1, gamma=0.9, epsilon = 0.05):

        env = gym.make(self.environment)
        Num_A = env.action_space.n
        Num_S = env.observation_space.n
        env.reset()

        NUM_ITER = num_iter

        Q = np.zeros((Num_S, Num_A))
        rewards = np.zeros(NUM_ITER)

        for it in range(1, NUM_ITER + 1):
            if it % 5000 == 0:
                print(f"Generating Episode Number: {it}")

            S = env.reset()
            A = self.epsilon_greedy_policy(Q[S], epsilon)
            
            G = 0
            step = 0
            while True:
                S_, R, terminal, _ = env.step(A)
                A_ = self.epsilon_greedy_policy(Q[S_], epsilon)
                
                G = G + (gamma**step)*R

                Q[S][A] = Q[S][A] + alpha * (R + gamma * Q[S_][A_] - Q[S][A])

                S = S_
                A = A_
                
                step += 1

                if terminal:
                    break
            rewards[it -1] = G

        return Q, rewards

    def show_policy(self, Q):

        MAX_STEPS = 51
        
        env = gym.make(self.environment)
        Num_A = env.action_space.n
        Num_S = env.observation_space.n

        S = env.reset()
        print(f"Starting state: {S}")

        step = 0
        done = False
        while step < MAX_STEPS:
            A = np.argmax(Q[S])
            env.render()
            S_, R, done, _ = env.step(A)
            if done:
                break
            print(
                f"Current State: {S}, action: {A}, reward: {R}, terminal: {done}, step: {step}"
            )
            S = S_
            step += 1
        if done:
            print(
                f"Current State: {S}, action: {A}, reward: {R}, terminal: {done}, step: {step}"
            )
            env.render()
        env.close()
        print("Finished", done)

In [13]:
agent = Agent(environment='CliffWalking-v0')

In [ ]:
q_value, rewards, policy = agent.on_policy_monte_carlo()
agent.show_policy(q_value)

In [6]:
off_policy, rewards = agent.off_policy_monte_carlo()
agent.show_policy(off_policy)

Generating Episode Number: 100
Generating Episode Number: 200
Generating Episode Number: 300
Generating Episode Number: 400
Generating Episode Number: 500
Starting state: 36
o  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
x  C  C  C  C  C  C  C  C  C  C  T

Current State: 36, action: 0, reward: -1, terminal: False, step: 0
o  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
x  o  o  o  o  o  o  o  o  o  o  o
o  C  C  C  C  C  C  C  C  C  C  T

Current State: 24, action: 1, reward: -1, terminal: False, step: 1
o  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
o  x  o  o  o  o  o  o  o  o  o  o
o  C  C  C  C  C  C  C  C  C  C  T

Current State: 25, action: 1, reward: -1, terminal: False, step: 2
o  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
o  o  x  o  o  o  o  o  o  o  o  o
o  C  C  C  C  C  C  C  C  C  C  T

Current State: 26, action: 1, reward: -1, terminal: False, st

In [7]:
q_policy, rewards = agent.q_learning()
agent.show_policy(q_policy)

Generating Episode Number: 5000
Generating Episode Number: 10000
Starting state: 36
o  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
x  C  C  C  C  C  C  C  C  C  C  T

Current State: 36, action: 0, reward: -1, terminal: False, step: 0
o  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
x  o  o  o  o  o  o  o  o  o  o  o
o  C  C  C  C  C  C  C  C  C  C  T

Current State: 24, action: 1, reward: -1, terminal: False, step: 1
o  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
o  x  o  o  o  o  o  o  o  o  o  o
o  C  C  C  C  C  C  C  C  C  C  T

Current State: 25, action: 1, reward: -1, terminal: False, step: 2
o  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
o  o  x  o  o  o  o  o  o  o  o  o
o  C  C  C  C  C  C  C  C  C  C  T

Current State: 26, action: 1, reward: -1, terminal: False, step: 3
o  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
o  o  o  x  o 

In [8]:
sarsa_policy, rewards = agent.sarsa()
agent.show_policy(sarsa_policy)

Generating Episode Number: 5000
Generating Episode Number: 10000
Starting state: 36
o  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
x  C  C  C  C  C  C  C  C  C  C  T

Current State: 36, action: 0, reward: -1, terminal: False, step: 0
o  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
x  o  o  o  o  o  o  o  o  o  o  o
o  C  C  C  C  C  C  C  C  C  C  T

Current State: 24, action: 0, reward: -1, terminal: False, step: 1
o  o  o  o  o  o  o  o  o  o  o  o
x  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
o  C  C  C  C  C  C  C  C  C  C  T

Current State: 12, action: 1, reward: -1, terminal: False, step: 2
o  o  o  o  o  o  o  o  o  o  o  o
o  x  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
o  C  C  C  C  C  C  C  C  C  C  T

Current State: 13, action: 0, reward: -1, terminal: False, step: 3
o  x  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o 